In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import datetime
import time
import predict
import base

In [2]:
#读取user_pay_count
user_pay_counts = base.ReadUserPayCountCSV();
#读取开店时间数据
shop_open_dates = base.ReadShopOpenDateCSV();
###读取商家信息
shop_infos = base.ReadShopInfoCSV();
##读取城市名称对照表
city_names = base.ReadCityNameCSV();
#读取休假表
calenders = base.ReadCalendarCSV();

In [3]:
#ARIMA

In [5]:
predict.initTrainArimasData('validation', shop_open_dates, user_pay_counts, pd.to_datetime('2016-10-7'), pd.to_datetime('2016-10-17'))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
#执行ArimaTrain.R

In [6]:
arima_df = predict.predictArimas('validation')

In [7]:
#ARIMA 时间序列预测
evaluations = {}


for id in range(1,2001):
    y_arima = arima_df.ix[id].values
    y_Test = base.getTestCount(user_pay_counts, id, pd.to_datetime('2016-10-18'), pd.to_datetime('2016-10-31')).values
    e = base.evaluation_tm(y_Test, y_arima)
    print("id=%d evaluation=%f"%(id, e))
    evaluations[id] = e
    
arima_evaluation_df = pd.DataFrame.from_dict(evaluations, orient='index');
arima_evaluation_df.columns = ['evaluation']
arima_evaluation_df.index.name = 'id'
arima_evaluation_df.sort_values('evaluation')
arima_evaluation_df[arima_evaluation_df['evaluation'].isnull()]
arima_evaluation_df = arima_evaluation_df.fillna(1)
arima_evaluation_df['evaluation'].mean()

id=1 evaluation=0.066773
id=2 evaluation=nan
id=3 evaluation=0.129179
id=4 evaluation=0.179509
id=5 evaluation=0.224783
id=6 evaluation=0.105457
id=7 evaluation=0.100879
id=8 evaluation=0.093823
id=9 evaluation=0.067390
id=10 evaluation=0.077807
id=11 evaluation=0.092153
id=12 evaluation=0.136097
id=13 evaluation=0.030201
id=14 evaluation=0.070064
id=15 evaluation=0.083871
id=16 evaluation=0.089043
id=17 evaluation=0.060542
id=18 evaluation=0.225443
id=19 evaluation=0.103596
id=20 evaluation=0.045817
id=21 evaluation=0.110406
id=22 evaluation=0.109483
id=23 evaluation=0.451271
id=24 evaluation=0.182771
id=25 evaluation=0.060576
id=26 evaluation=0.213390
id=27 evaluation=0.055413
id=28 evaluation=0.107315
id=29 evaluation=0.144746
id=30 evaluation=0.078763
id=31 evaluation=0.143498
id=32 evaluation=0.215788
id=33 evaluation=0.129003
id=34 evaluation=0.080249
id=35 evaluation=0.154304
id=36 evaluation=0.079828
id=37 evaluation=0.074249
id=38 evaluation=0.140188
id=39 evaluation=0.132449


0.12318842791610882

In [8]:
arima_evaluation_df.to_csv("./data/dataset/dataset/validation/arima_evaluation.txt", sep='\t', index=True, encoding='UTF-8')

In [ ]:
#Holiday-Mean

In [9]:
holiday_mean_df = predict.trainHolidayMeansData(
    shop_open_dates, user_pay_counts, calenders, pd.to_datetime('2016-10-10'), pd.to_datetime('2016-10-17'))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [11]:
start_time = pd.to_datetime('10/18/2016');
end_time = pd.to_datetime('10/31/2016');
predict_date = pd.date_range(start=start_time, end=end_time, freq='D', normalize=True)
pridicts = {}
evaluations = {}
for id in range(1, 2001):
    y_Pred = []
    for date in predict_date:
        daytype = calenders.loc[date.strftime('%Y-%m-%d')]['daytype']
        y_Pred.append(holiday_mean_df.loc[id][daytype]);
    pridicts[id] = y_Pred
    y_Test = base.getTestCount(user_pay_counts, id, start_time, end_time).values
    e = base.evaluation(y_Test, y_Pred)
    print("id=%d evaluation=%f"%(id, e))
    pridicts[id] = y_Pred
    evaluations[id] = e

id=1 evaluation=0.063019
id=2 evaluation=nan
id=3 evaluation=0.085672
id=4 evaluation=0.100392
id=5 evaluation=0.219213
id=6 evaluation=0.099061
id=7 evaluation=0.065653
id=8 evaluation=0.096082
id=9 evaluation=0.041169
id=10 evaluation=0.085147
id=11 evaluation=0.081634
id=12 evaluation=0.102969
id=13 evaluation=0.032192
id=14 evaluation=0.083759
id=15 evaluation=0.054236
id=16 evaluation=0.101319
id=17 evaluation=0.040555
id=18 evaluation=0.231084
id=19 evaluation=0.068369
id=20 evaluation=0.039045
id=21 evaluation=0.057741
id=22 evaluation=0.078145
id=23 evaluation=0.409551
id=24 evaluation=0.174239
id=25 evaluation=0.043172
id=26 evaluation=0.078091
id=27 evaluation=0.087238
id=28 evaluation=0.069608
id=29 evaluation=0.069696
id=30 evaluation=0.090506
id=31 evaluation=0.076062
id=32 evaluation=0.050783
id=33 evaluation=0.069542
id=34 evaluation=0.023106
id=35 evaluation=0.111814
id=36 evaluation=0.066690
id=37 evaluation=0.042448
id=38 evaluation=0.099575
id=39 evaluation=0.123689


In [12]:
evaluation_df = pd.DataFrame.from_dict(evaluations, orient='index');
evaluation_df.columns = ['evaluation']
evaluation_df.index.name = 'id'

In [13]:
evaluation_df = evaluation_df.fillna(1)
evaluation_df.to_csv("./data/dataset/dataset/validation/holiday_mean_evaluation.txt", sep='\t', index=True, encoding='UTF-8')

In [14]:
evaluation_df['evaluation'].mean()

0.09116614771512238